In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove

In [2]:
# The smart contract as a transition system.
ts = Ts('auction')

addrSort = z3.BitVecSort(16)

auctionEndTime = z3.Int('auctionEndTime')

ended, endedOut = ts.add_var(z3.BoolSort(), name='ended')

pendingReturns, pendingReturnsOut = ts.add_var(z3.ArraySort(addrSort, z3.BitVecSort(16)),
        name= 'pendingReturns')

highestBidder, highestBidderOut = ts.add_var(addrSort, name='highestBidder')

highestBid, highestBidOut = ts.add_var(z3.BitVecSort(16))

withdrawCounts, withdrawCountsOut = ts.add_var(z3.ArraySort(addrSort, z3.IntSort()), 
        name='withdrawCount')

In [3]:
p = z3.BitVec('p',16)
ts.Init = z3.And(z3.ForAll(p,pendingReturns[p]==0),
        highestBid == 0,
        highestBidder == 0,
        ended == False,
        z3.ForAll(p, withdrawCounts[p]==0)
)

In [4]:
now = z3.Int('now')
value = z3.BitVec('value', 16)
sender = z3.BitVec('sender', 16)

tr_bid = z3.And(now < auctionEndTime,
        value > highestBid,
        z3.Not(ended),
        z3.Or(
            z3.And(highestBid != 0, 
                pendingReturnsOut == z3.Update(pendingReturns,highestBidder,
                                        pendingReturns[highestBidder]+highestBid)),
            z3.And(highestBid == 0, pendingReturnsOut==pendingReturns)
        ),
        highestBidOut == value,
        highestBidderOut == sender,
        withdrawCountsOut == withdrawCounts,
        endedOut == ended,
        )

tr_withdraw = z3.And(
        pendingReturns[sender]>0,
        ended,
        pendingReturnsOut == z3.Update(pendingReturns, sender, 0),
        withdrawCountsOut == z3.Update(withdrawCounts, sender, withdrawCounts[sender]+1),
        endedOut == ended,
        highestBidderOut==highestBidder,
        highestBidOut == highestBid
)

tr_end = z3.And(
        now >= auctionEndTime,
        z3.Not(ended),
        endedOut == True,
        pendingReturnsOut == pendingReturns,
        withdrawCountsOut==withdrawCounts,
        highestBidderOut==highestBidder,
        highestBidOut == highestBid
)

ts.Tr = z3.Or( 
    tr_bid, 
    tr_withdraw, 
    tr_end
    )

In [5]:
p1 = z3.ForAll(p, z3.Implies(pendingReturns[p] > 0, withdrawCounts[p] == 0))
p2 = z3.ForAll(p, z3.Implies(z3.And(z3.Not(ended)), withdrawCounts[p] == 0))

In [6]:
prove_inductive(ts, z3.And(p1,p2))

Prove init => property.
proved
Prove property is inductive.
proved


In [7]:
p3 = z3.ForAll(p, z3.Implies(pendingReturns[p]<=0, withdrawCounts[p] <= 1))

In [8]:
prove_inductive(ts, z3.And(p1,p2,p3))

Prove init => property.
proved
Prove property is inductive.
proved


In [9]:
ts_property = z3.ForAll(p, withdrawCounts[p] <= 1)
prove(z3.Implies(z3.And(p1,p2,p3), ts_property))

proved


[Not(Implies(And(ForAll(p,
                        Implies(pendingReturns[p] > 0,
                                withdrawCount[p] == 0)),
                 ForAll(p,
                        Implies(And(Not(ended)),
                                withdrawCount[p] == 0)),
                 ForAll(p,
                        Implies(pendingReturns[p] <= 0,
                                withdrawCount[p] <= 1))),
             ForAll(p, withdrawCount[p] <= 1)))]